In [37]:
#Cargar textos para el analisis vector Test desde MongoDB
from pymongo import MongoClient

# Create a client instance of MongoClient
client= MongoClient('mongodb://localhost:27017/')
db= client.cultura
collection  = db.prueba1
# Convertir a Panda
import pandas as pd

data = pd.DataFrame(list(collection.find()))
data_kNN = data
#Eliminar valores nan de la columna tweet
data['tweet'] = data['tweet'].fillna(' ')

In [38]:
data.head(5)

,_id,lat,lon,timestamp,tweet,user
0,5d1b8c3a5923dbcca28bd256,37.9774,-1.063,Tue Apr 15 02:23:01 +0000 2014,Ahora mismo si tuviese una pistola me pegaba u...,@AlexLuna_72
1,5d1b8c3a5923dbcca28bd257,,,Tue Apr 15 02:23:22 +0000 2014,Hola. Visto 05:36. Ms que Visto 05:38. I...,@Gissell_Tytta
2,5d1b8c3a5923dbcca28bd258,37.9798,-1.06197,Tue Apr 15 02:23:32 +0000 2014,Buenas noches a todos... Fav ;) juju,@69Rikifriki
3,5d1b8c3a5923dbcca28bd259,,,Tue Apr 15 02:24:44 +0000 2014,JAJAJA,@Gissell_Tytta
4,5d1b8c3a5923dbcca28bd25a,38.0688,-1.19224,Tue Apr 15 02:25:22 +0000 2014,Salimos para a corua :),@AndreaGalian


In [39]:
#Borramos las columnas que no nos interesan
data = data.drop(['_id','lat','lon','timestamp','user'],axis=1)
#Creamos una columna para el procesado y comparar texto al final
data['tweets_procesados'] = data['tweet']

In [40]:
data.head(5)

,tweet,tweets_procesados
0,Ahora mismo si tuviese una pistola me pegaba u...,Ahora mismo si tuviese una pistola me pegaba u...
1,Hola. Visto 05:36. Ms que Visto 05:38. I...,Hola. Visto 05:36. Ms que Visto 05:38. I...
2,Buenas noches a todos... Fav ;) juju,Buenas noches a todos... Fav ;) juju
3,JAJAJA,JAJAJA
4,Salimos para a corua :),Salimos para a corua :)


In [41]:
#Funcion Limpieza de mensajes

######Limpieza General############
import re
from unicodedata import normalize

# -> NFD y eliminar diacríticos
#x = tweets_corpus.content.get_values()
#longitud=len(tweets_corpus.content.get_values())

def limpieza(x):
    for i in range(0,len(x)):
        x[i] = str(x[i])
        x[i]= re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD",x[i]), 0, re.I
            )
        x[i]=re.sub('(f|ht)tp(s?)://(.*)[.][a-z]+',' ',x[i])
        x[i]=re.sub('(f|ht)tp(s?)://(.*)[.][a-z]+',' ',x[i])
        x[i]=re.sub('[a-zA-Z0-9.?{}]+@\w+\.\w+.\w*','',x[i])
        x[i]=re.sub('\[[a-zA-Z0-9\,\. ]+\]','',x[i]) 
        x[i]=re.sub('\([a-zA-Z0-9\,\.\- ]+\)',' ',x[i])
        x[i]=re.sub('((et al\.)|(i\.i\.d\.)|(i\.e\.)|\-|\'|\’|\`)','',x[i])
        x[i]=re.sub('[^a-zA-Z_á\éíóúà\èìòùäëïöü\s]','',x[i])    
        x[i]=re.sub(' +',' ',x[i])    
        x[i]=re.sub('(a-z|A-Z){1,1}','',x[i].lower()) #Convertir a minuscula y quitar caracteres
    return x


In [42]:
##Limpiar mensajes
data.content=limpieza(data['tweets_procesados'].get_values())
data.head(5)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,tweet,tweets_procesados
0,Ahora mismo si tuviese una pistola me pegaba u...,ahora mismo si tuviese una pistola me pegaba u...
1,Hola. Visto 05:36. Ms que Visto 05:38. I...,hola visto ms que visto ignorado no le importas
2,Buenas noches a todos... Fav ;) juju,buenas noches a todos fav juju
3,JAJAJA,jajaja
4,Salimos para a corua :),salimos para a corua


In [43]:
#Tokenizado y stemmer funciones
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')    

def tokenizar(x):
    for i in range(0,len(x)): 
        #Tokenizar
        x[i] = x[i].split(' ')    
    return x

def stemming (x):
    for i in range(0,len(x)):   
        #Stemm
        x[i]=[stemmer.stem(elemento) for elemento in x[i]] 
    return x

def reconstruir_texto (x):
    for i in range(0,len(x)):   
        #Recuperacion de texto con ajustes
        x[i] = ' '.join(x[i])
    return x

In [44]:
# Tokenizar y stemmer
tokenizado = tokenizar(data['tweets_procesados'].get_values())
data['tokenizados'] = tokenizado
stemmizado = stemming(tokenizado)
data['tweets_procesados'] = reconstruir_texto(stemmizado)
data.head(5)

,tweet,tweets_procesados,tokenizados
0,Ahora mismo si tuviese una pistola me pegaba u...,ahor mism si tuv una pistol me peg un tir sin ...,"[ahora, mismo, si, tuviese, una, pistola, me, ..."
1,Hola. Visto 05:36. Ms que Visto 05:38. I...,hol vist ms que vist ignor no le import,"[hola, visto, ms, que, visto, ignorado, no, le..."
2,Buenas noches a todos... Fav ;) juju,buen noch a tod fav juju,"[buenas, noches, a, todos, fav, juju]"
3,JAJAJA,jajaj,[jajaja]
4,Salimos para a corua :),sal par a coru,"[salimos, para, a, corua, ]"


In [45]:
###Cargamos clasificador SVM previamente entrenado
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from nltk.corpus import stopwords
from sklearn import svm
spanish_stopwords = stopwords.words('spanish')

#Cargamos clasificador
clf_load = joblib.load('SVM_entrenado_sentimientos_tres_estados.pkl') 
#clf_load = joblib.load('SVM_entrenado_sentimientos_binario.pkl') 

# Cargar vectorizador de entrenamiento
vectorizer = joblib.load('vectorizado_tres_estados.pkl')
#vectorizer = joblib.load('vectorizado_binario.pkl')

# Vectorizar texto
mensajes = vectorizer.transform(data['tweets_procesados'])

#Realizar clasficiacion
data['polaridad_SVM'] = clf_load.predict(mensajes)


In [46]:
data.head(5)

,tweet,tweets_procesados,tokenizados,polaridad_SVM
0,Ahora mismo si tuviese una pistola me pegaba u...,ahor mism si tuv una pistol me peg un tir sin ...,"[ahora, mismo, si, tuviese, una, pistola, me, ...",-1
1,Hola. Visto 05:36. Ms que Visto 05:38. I...,hol vist ms que vist ignor no le import,"[hola, visto, ms, que, visto, ignorado, no, le...",1
2,Buenas noches a todos... Fav ;) juju,buen noch a tod fav juju,"[buenas, noches, a, todos, fav, juju]",1
3,JAJAJA,jajaj,[jajaja],1
4,Salimos para a corua :),sal par a coru,"[salimos, para, a, corua, ]",1


In [47]:
### Ahora por KNN
#clf_load_knn = joblib.load('KNN_entrenado_sentimientos_binario.pkl')
clf_load_knn = joblib.load('KNN_entrenado_sentimientos_tres_estados.pkl')
data['polaridad_KNN']  = clf_load_knn.predict(mensajes)

In [48]:
data.sample(7)

,tweet,tweets_procesados,tokenizados,polaridad_SVM,polaridad_KNN
7025,me piro,me pir,"[me, piro]",1,1
2445,Que pesaos con la final,que pesa con la final,"[que, pesaos, con, la, final]",1,1
2095,"Preparando los zarageles,chaleco ect y el hgad...",prepar los zarageleschalec ect y el hgad pa st...,"[preparando, los, zarageleschaleco, ect, y, el...",1,1
7811,http://t.co/6o8WHkFRsf,owhkfrsf,"[, owhkfrsf]",1,1
8932,morning&gt;&gt;,morninggtgt,[morninggtgt],1,1
3402,@TinuMiReina neverrrr,tinumirein neverrrr,"[tinumireina, neverrrr]",1,1
5838,No echaaaan naa en la tele,no echaa naa en la tel,"[no, echaaaan, naa, en, la, tele]",1,1


In [49]:
## Exportando la información a excel
data.to_excel(r'sentimientosSVM&KNN_tweets.xlsx')

In [50]:
# LLevar los resultados a una base de datos de MongoDB
collist = db.list_collection_names()

## ALMACENAR RESULTADOS SVM ###
if "sentimientos" in collist:
    db.sentimientos.drop()   
    db.sentimientos.insert_many(data.to_dict('records'))
else:
    db["sentimientos"]
    db.sentimientos.insert_many(data.to_dict('records'))

In [51]:
#clf_load.predict(mensajes)